In [1]:
from langchain.chat_models import init_chat_model
from langchain_postgres import PGVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.tools import tool
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
if not os.environ.get("GOOGLE_API_KEY"):
  print("Please set the GOOGLE_API_KEY environment variable.")

model = init_chat_model("google_genai:gemini-2.5-flash")

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [3]:
if not os.environ.get("SUPABASE_CONNECTION_STRING"):
    os.environ["SUPABASE_CONNECTION_STRING"] = getpass.getpass("Enter connection string for Supabase: ")

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="lilianweng_blog",
    connection=os.environ["SUPABASE_CONNECTION_STRING"],
)

In [ ]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [9]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a collection of blog posts. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [10]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    last_message = event["messages"][-1]
    last_message.pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=95f51f27-938c-4aeb-ba40-ec64bca46c4a; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=4ef27f44-9eb1-490a-a795-4875314a50cd; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=eb5714b0-5806-4119-9f33-47ba32ff2e6f


================================== Ai Message ==================================
Tool Calls:
  retrieve_context (4c56a5aa-9639-4e6a-97e7-86d6a426fda6)
 Call ID: 4c56a5aa-9639-4e6a-97e7-86d6a426fda6
  Args:
    query: standard method for Task Decomposition


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=eb5714b0-5806-4119-9f33-47ba32ff2e6f; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=4ef27f44-9eb1-490a-a795-4875314a50cd; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=afe9cbbf-4239-4c1e-a08f-1bcb50cbd735; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=e783587b-c609-4d9f-8fb1-39bc513ea0a0


================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step 

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=e783587b-c609-4d9f-8fb1-39bc513ea0a0; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=afe9cbbf-4239-4c1e-a08f-1bcb50cbd735; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=1e90a27d-aa41-47f1-b4a6-a4e8ca353951; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=997ceb4a-9384-4d24-b7a7-5c43eee4704c


================================== Ai Message ==================================

[{'type': 'text', 'text': 'The standard method for Task Decomposition can be achieved in a few ways:\n\n1.  **By LLM with simple prompting**: This involves using a Large Language Model (LLM) with straightforward prompts such as "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?".\n2.  **By using task-specific instructions**: This involves providing specific instructions tailored to the task, for example, "Write a story outline." for a novel writing task.\n3.  **With human inputs**: This method incorporates human guidance in the decomposition process.\n\nA common extension of these methods, particularly for long-horizon planning, is **LLM+P (Liu et al. 2023)**. This approach relies on an external classical planner. It uses the Planning Domain Definition Language (PDDL) as an intermediate interface. In this process, the LLM translates the problem into "Problem PDDL", then requests a classica

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=997ceb4a-9384-4d24-b7a7-5c43eee4704c; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=1e90a27d-aa41-47f1-b4a6-a4e8ca353951; trace=95f51f27-938c-4aeb-ba40-ec64bca46c4a,id=95f51f27-938c-4aeb-ba40-ec64bca46c4a


In [ ]:
# # Render the agent's last message as Markdown in the Jupyter output
from IPython.display import Markdown, display
display(Markdown(last_message.text))

The standard method for Task Decomposition can be achieved in a few ways:

1.  **By LLM with simple prompting**: This involves using a Large Language Model (LLM) with straightforward prompts such as "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?".
2.  **By using task-specific instructions**: This involves providing specific instructions tailored to the task, for example, "Write a story outline." for a novel writing task.
3.  **With human inputs**: This method incorporates human guidance in the decomposition process.

A common extension of these methods, particularly for long-horizon planning, is **LLM+P (Liu et al. 2023)**. This approach relies on an external classical planner. It uses the Planning Domain Definition Language (PDDL) as an intermediate interface. In this process, the LLM translates the problem into "Problem PDDL", then requests a classical planner to generate a PDDL plan based on an existing "Domain PDDL", and finally translates the PDDL plan back into natural language. This essentially outsources the planning step to an external tool, which is common in certain robotic setups.